This file demonstrates how we can obtain the (univariate) regression coefficients very quickly, without having to use any loop, between EACH feature in an input matrix X and the response variable y.

We first estimate the "reverse" model (y->X), i.e. y regressed onto X, which is algebraically straightforward to compute. This model will give us $\beta_{y->x}$ (and \alpha_{y->x}, which we don't need). 

Then we use the (mathematical) relation betwen both univariate regressions to compute the coefficients from  regressing each column of X onto y as follows:

$$\beta_{x->y} = \beta_{y->x} \cdot \left(\frac{\sigma_y}{\sigma_x}\right)^2$$
$$\alpha_{x->y} = \mu_y -\mu_x \cdot  \beta_{y->x}  \cdot \left(\frac{\sigma_y}{\sigma_x}\right)^2$$



In [1]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import time

In [2]:
X,y = datasets.make_regression(n_features=int(1e5), random_state=69)

In [3]:
def compute_brute(X,y):
    """
    
    This function just loops through each feature
    
    """
    LinReg = LinearRegression()
    alpha, beta = [],[]
    for x in X.T:
        LinReg.fit(x[:,None], y)
        alpha.append(LinReg.intercept_)
        beta.append(LinReg.coef_[0])
    
    alpha = np.array(alpha)
    beta = np.array(beta)

    return alpha, beta
    
t_0 = time.time()
alpha_brute, beta_brute = compute_brute(X,y)
t_f = time.time()
print("brute force time = %f s" % (t_f-t_0))

brute force time = 34.893205 s


In [4]:
def compute_inverse(X, y):
    """
    
    This function computes first the regression of y onto 
    the whole matrix of features, and then use the estimated
    coefficients to calculate the coefficients from regressing
    each column of X onto y
    
    """
    
    LinReg = LinearRegression()
    LinReg.fit(y[:, None], X)
    beta_inv = LinReg.coef_.flatten()

    varx = np.var(X, axis=0)
    vary = np.var(y)
    mux = np.mean(X, axis=0)
    muy = np.mean(y)

    beta = (vary*beta_inv)/varx
    alpha = muy - (mux*beta_inv/varx)*vary
    return alpha, beta

t_0 = time.time()
alpha_inv, beta_inv = compute_inverse(X,y)
t_f = time.time()
print("inverse force time = %f s" % (t_f-t_0))

inverse force time = 0.195947 s


In [5]:
# Both solutions should be the same
print("correlation of the intercepts between both approaches= %f" % np.corrcoef(alpha_brute, alpha_inv)[0,1])
print("correlation of the betas between both  = %f" %np.corrcoef(beta_brute.flatten(), beta_inv)[0,1])

correlation of the intercepts between both approaches= 1.000000
correlation of the betas between both  = 1.000000
